In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import re
import nltk
from nltk.tokenize import TweetTokenizer
import string
import unidecode

import warnings
warnings.filterwarnings('ignore')

stopwords = nltk.corpus.stopwords.words('english')

In [2]:
tweets = pd.read_csv("../data/train.csv")

In [3]:
def concatenate(x,char):
    words = ""
    for word in x:
        if word.startswith(char):
            words = words + word + " "
    return words

def count_vowels(x):
    return (x.count('a') + x.count('e') + x.count('i') + x.count('o') + x.count('u'))

def count_short_words(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if 1 <= len(word) <= 3:
            count += 1
    return count

def count_stopwords(x):
    count = 0
    words = x.split(' ')
    for word in words:
        if word in stopwords:
            count += 1
    return count

In [4]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [5]:
tweets["special_chars_count"] =  tweets["text"]
tweets["special_chars_count"] =  tweets["special_chars_count"].str.lower()
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
tweets["special_chars_count"] = tweets["special_chars_count"].str.strip()
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(' +','', x))
tweets["special_chars_count"] = tweets["special_chars_count"].apply(lambda x: re.sub(r'[0-9]','', x))
tweets["special_chars_count"] = tweets["special_chars_count"].str.len()

tweets["hashtags"] = tweets["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'#'))
tweets["labels"] = tweets["text"].str.lower().str.split(' ').apply(lambda x: concatenate(x,'@'))
tweets["hashtags_count"] = tweets["hashtags"].str.split(' ').apply(lambda x: len(x))-1
tweets["labels_count"] = tweets["labels"].str.split(' ').apply(lambda x: len(x))-1

tweets["num_chars_count"] = tweets["text"]
tweets["num_chars_count"] =  tweets["num_chars_count"].str.lower()
tweets["num_chars_count"] = tweets["num_chars_count"].apply(lambda x: re.sub(r'[a-z]','',x))
tweets["num_chars_count"] = tweets["num_chars_count"].apply(lambda x: re.sub(r'[^\w]','',x))
tweets["num_chars_count"] = tweets["num_chars_count"].str.strip()
tweets["num_chars_count"] = tweets["num_chars_count"].str.len()

tweets["clean_text"] = tweets["text"].apply(lambda x: cleaning_text(x))

tweets["text"] = tweets["text"].str.lower()
tweets["text"] = tweets["text"].apply(lambda x: re.sub('(?P<url>https?://[^\s]+)', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'[^\w]', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'_', ' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'[0-9]',' ', x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(' +',' ', x))
tweets["text"] = tweets["text"].apply(lambda x: unidecode.unidecode(x))
tweets["text"] = tweets["text"].str.strip()
tweets["text_length"] = tweets["text"].str.len()

tweets["vowels_count"] = tweets["text"].apply(lambda x: count_vowels(x))
tweets["short_words_count"] = tweets["text"].apply(lambda x: count_short_words(x))
tweets["stopwords_count"] = tweets["text"].apply(lambda x: count_stopwords(x))
tweets["text"] = tweets["text"].apply(lambda x: re.sub(r'\b\w{1}\b', '', x))
tweets["words_count"] = tweets["text"].str.split(' ').apply(lambda x: len(x))

tweets.rename(columns={"target":"target_label"}, inplace=True)
tweets.head()

,id,keyword,location,text,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,clean_text,text_length,vowels_count,short_words_count,stopwords_count,words_count
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,,1,0,0,deeds reason earthquake may allah forgive us,68,25,7,6,13
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,,,0,0,0,forest fire near la ronge sask canada,37,13,1,0,7
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,,,0,0,0,residents asked shelter place notified officer...,130,45,9,11,22
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,,1,0,5,people receive wildfires evacuation orders cal...,56,24,1,1,7
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,,2,0,0,got sent photo ruby alaska smoke wildfires pou...,85,25,3,7,16


In [6]:
tweets["keyword"] = tweets["keyword"].str.replace('%20',' ')
tweets["keyword"] = tweets["keyword"].astype('category')

In [7]:
#one hot encoding
dummies = pd.get_dummies(tweets["keyword"], prefix="keyword")
dummies.columns

Index(['keyword_ablaze', 'keyword_accident', 'keyword_aftershock',
       'keyword_airplane accident', 'keyword_ambulance', 'keyword_annihilated',
       'keyword_annihilation', 'keyword_apocalypse', 'keyword_armageddon',
       'keyword_army',
       ...
       'keyword_weapons', 'keyword_whirlwind', 'keyword_wild fires',
       'keyword_wildfire', 'keyword_windstorm', 'keyword_wounded',
       'keyword_wounds', 'keyword_wreck', 'keyword_wreckage',
       'keyword_wrecked'],
      dtype='object', length=221)

In [8]:
tweets_ohe = pd.concat([tweets,dummies], axis="columns")
tweets_ohe.shape

(7613, 238)

In [9]:
#BOW
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(tweets["clean_text"])
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [10]:
feature_words = vectorizer.get_feature_names()
df_words = pd.DataFrame(X.toarray(), columns=feature_words)
df_words.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,...,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_filter = df_words.loc[:,(df_words.sum()>5)]
df_filter.shape

(7613, 2080)

In [12]:
tweets_final = pd.concat([tweets_ohe,df_filter], axis="columns")
tweets_final.shape

(7613, 2318)

In [13]:
X = tweets_final.drop(["id","keyword","location","text","target_label","hashtags","labels","clean_text"], axis=1)
y = tweets_final["target_label"]

In [14]:
print(X.shape)
print(y.shape)

(7613, 2308)
(7613,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [17]:
#usando las 2308 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [18]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.785464


In [19]:
df_feat_importances = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [20]:
df_feat_importances.head()

,importancia
text_length,102
special_chars_count,88
vowels_count,84
stopwords_count,70
short_words_count,57


In [21]:
list_fi = df_feat_importances.index[:800].tolist()
X = X.filter(items=list_fi)
X.columns

Index(['text_length', 'special_chars_count', 'vowels_count', 'stopwords_count',
       'short_words_count', 'num_chars_count', 'words_count', 'hiroshima',
       'labels_count', 'fires',
       ...
       'uam', 'typhoondevastated', 'type', 'twister', 'twice', 'tweets',
       'tweet', 'tv', 'turn', 'uiwhen'],
      dtype='object', length=800)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [90]:
#usando 800 features
model_lgb = lgb.LGBMClassifier(n_estimators=100, colsample_bytree=0.5, num_leaves=53, objective='binary', max_depth=21)
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=21,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=53, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [91]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.790280


In [156]:
model_lgb = lgb.LGBMClassifier(n_estimators=500, colsample_bytree=0.7, num_leaves=14, objective='binary', max_depth=7,
                              min_child_samples=20, min_child_weight=0.05)
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=20, min_child_weight=0.05, min_split_gain=0.0,
               n_estimators=500, n_jobs=-1, num_leaves=14, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [157]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.795972


In [162]:
#ahora con tf-idf
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(tweets["clean_text"])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [163]:
df_words = pd.DataFrame(X.toarray(), columns=feature_words)
df_filter = df_words.loc[:,(df_words.sum()>2)]
df_filter.shape

(7613, 2072)

In [165]:
tweets_final = pd.concat([tweets_ohe,df_filter], axis="columns")
tweets_final.shape

(7613, 2310)

In [335]:
X = tweets_final.drop(["id","keyword","location","text","target_label","hashtags","labels","clean_text"], axis=1)
y = tweets_final["target_label"]

In [336]:
X.shape

(7613, 2300)

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [169]:
#usando las 2300 features
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [170]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.781524


In [171]:
df_feat_importances = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [181]:
df_feat_importances["importancia"].value_counts()

0     1931
1       43
2       34
7       33
6       31
3       31
4       29
5       27
8       20
11      17
9       16
10      16
14      13
12      10
13       7
17       6
15       5
16       4
18       4
26       4
21       3
23       2
25       2
19       2
47       2
31       1
35       1
60       1
30       1
54       1
70       1
27       1
69       1
Name: importancia, dtype: int64

In [182]:
2300-1931

369

In [329]:
list_fi = df_feat_importances.index[:800].tolist()
X = X.filter(items=list_fi)
X.shape

(7613, 800)

In [330]:
df_feat_importances.head(30)

,importancia
vowels_count,70
text_length,69
special_chars_count,60
stopwords_count,54
num_chars_count,47
short_words_count,47
hiroshima,35
fires,31
labels_count,30
people,27


In [178]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [256]:
#usando 800 features
model_lgb = lgb.LGBMClassifier(num_leaves=14, objective="binary", colsample_bytree=0.6, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=150, min_child_samples=10, reg_lambda=1, learning_rate=0.2)
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.6,
               importance_type='split', learning_rate=0.2, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=150, n_jobs=1, num_leaves=14, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=1, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [257]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.791594


In [258]:
model_lgb.feature_importances_

array([69, 70, 77, 68, 53, 65, 15, 18, 21, 13, 11, 49, 11, 15, 11, 11, 15,
        9,  9,  9, 12,  8,  8,  9,  5,  8,  9,  9,  8,  8,  6,  5,  9,  9,
        6,  5,  7, 11,  9, 10,  9,  9,  6,  5,  6,  6,  7,  8,  8,  4,  0,
        7,  6,  8,  4,  8,  6,  5,  7,  5,  7,  7,  9,  6,  5,  7,  8,  7,
        4,  7,  5,  6,  0,  5,  5, 10,  5,  5,  4,  4,  7,  2,  4,  5,  4,
        6,  4,  5,  5,  4,  4,  4,  4,  3,  4,  7,  3,  3,  5,  3,  4,  4,
        2, 10,  5,  3,  3,  4,  2,  4,  5,  5,  5,  4,  4,  4,  1,  3,  1,
        4,  3,  5,  2,  2,  6,  3,  4,  1,  4,  1,  2,  2,  3,  3,  3,  2,
        4,  1,  5,  2,  3,  2,  2,  3,  4,  1,  3,  2,  3,  3,  3,  1,  2,
        4,  7,  3,  6,  3,  4,  3,  4,  3,  6,  3,  3,  4,  1,  4,  2,  4,
        3,  3,  3,  4,  0,  2,  1,  1,  2,  3,  4,  2,  1,  3,  3,  3,  5,
        0,  1,  1,  5,  3,  1,  0,  1,  3,  2,  1,  1,  3,  1,  3,  3,  2,
        3,  1,  2,  2,  1,  2,  2,  1,  3,  2,  1,  3,  2,  2,  0,  0,  0,
        4,  1,  1,  0,  0

In [259]:
df_feat_importances2 = pd.DataFrame(model_lgb.feature_importances_, index=X_train.columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

In [337]:
df_feat_importances2["importancia"].value_counts().nlargest()

0    492
1     72
2     51
3     46
4     39
Name: importancia, dtype: int64

In [338]:
list_fi = df_feat_importances2.index[:308].tolist()
X = X.filter(items=list_fi)
X.shape

(7613, 308)

In [339]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [382]:
#usando 308 features
model_lgb = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=250, min_child_samples=10, reg_lambda=1, learning_rate=0.2)
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.2, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=250, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=1, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [383]:
y_test_hat = model_lgb.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.790718
